In [10]:
%load_ext autoreload
%autoreload 2
import gc
import numpy as np
import pandas as pd
import sys
import re
from glob import glob
import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
#  sys.path.append(f"/mnt/c/Git/go/kaggle/github/library/")
import utils
from utils import logger_func, mkdir_func
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

import eda
from utils import get_categorical_features, get_numeric_features, parallel_process
from feature_engineering import diff_feature, division_feature, product_feature, cnt_encoding, select_category_value_agg, exclude_feature, target_encoding
from tqdm import tqdm

#========================================================================
# Global Variable
sys.path.append(f"../py")
from info_home_credit import hcdr_key_cols
key, target, ignore_list = hcdr_key_cols()
#========================================================================

base = utils.read_df_pkl('../input/base0*').set_index(key)

100%|██████████| 3/3 [00:00<00:00, 553.36it/s]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
app = utils.read_df_pkl(path='../input/clean_app*.p')
display(app.columns)


100%|██████████| 3/3 [00:01<00:00,  2.00it/s]


Index(['AMT_ANNUITY', 'AMT_CREDIT', 'AMT_GOODS_PRICE', 'AMT_INCOME_TOTAL',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_YEAR',
       ...
       'TARGET', 'TOTALAREA_MODE', 'WALLSMATERIAL_MODE',
       'WEEKDAY_APPR_PROCESS_START', 'YEARS_BEGINEXPLUATATION_AVG',
       'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BEGINEXPLUATATION_MODE',
       'YEARS_BUILD_AVG', 'YEARS_BUILD_MEDI', 'YEARS_BUILD_MODE'],
      dtype='object', length=122)

In [45]:
num_list = get_numeric_features(app, ignore_list)
amt_list = [col for col in app.columns if col.count('AMT_') and not(col.count('BUREAU'))]
day_list = [col for col in app.columns if col.count('DAYS_')]
other_list = list(set(num_list) - set(amt_list) - set(day_list))

In [ ]:
#========================================================================
# Feature Engineering 
df = app
cat_enc = CategoricalEncoding(data=df, key=key, target=target, ignore_list=[key, target], base=base)
prefix = 'f000_app_'
cat_list = get_categorical_features(df, ignore_list)
df_ohe = cat_enc.go_ohe(cat_list=cat_list, method_list=['sum', 'mean'], prefix=prefix)
df_label = cat_enc.go_label(cat_list=cat_list, method_list=['mean', 'std', 'max', 'min'], prefix=prefix)
#========================================================================

In [5]:
prefix = 'f000_app_'
df = app
org_col_list = df.columns.tolist()

amt_cre = 'AMT_CREDIT'
amt_ann = 'AMT_ANNUITY'
amt_inc = 'AMT_INCOME_TOTAL'
day_bir = 'DAYS_BIRTH'
day_emp = 'DAYS_EMPLOYED'
day_pub = 'DAYS_ID_PUBLISH'
day_pho = 'DAYS_LAST_PHONE_CHANGE'
day_reg = 'DAYS_REGISTRATION'


div_list = [
    [amt_cre, amt_ann]
    ,[amt_cre, amt_inc]
    ,[amt_ann, amt_inc]
    ,[amt_cre, day_bir]
    ,[amt_inc, day_bir]
    ,[amt_ann, day_bir]
    ,[amt_cre, day_emp]
    ,[amt_inc, day_emp]
    ,[amt_ann, day_emp]
    ,[amt_cre, day_pub]
    ,[amt_inc, day_pub]
    ,[amt_ann, day_pub]
    ,[amt_cre, day_pho]
    ,[amt_inc, day_pho]
    ,[amt_ann, day_pho]
    ,[amt_cre, day_reg]
    ,[amt_inc, day_reg]
    ,[amt_ann, day_reg]
]

diff_list = [
    [amt_ann, amt_inc]
    ,[day_bir, day_emp]
    ,[day_bir, day_pub]
    ,[day_bir, day_pho]
    ,[day_bir, day_reg]
    ,[day_emp, day_pub]
    ,[day_emp, day_pho]
    ,[day_emp, day_reg]
    ,[day_pub, day_pho]
    ,[day_pub, day_reg]
    ,[day_pho, day_reg]
]

for (f1, f2) in div_list:
    df[f"{f1}_per_{f2}"] = df[f1] / (df[f2]+1) 
    
for (f1, f2) in diff_list:
    df[f"{f1}_diff_{f2}"] = df[f1] / (df[f2]+1) 
    
df[f"{amt_cre}-per-EMP_TERM"]     = df[amt_cre] - (df[day_bir] - df[day_emp])
df[f"{amt_cre}-per-PUBLISH_TERM"] = df[amt_cre] - (df[day_bir] - df[day_pub])
df[f"{amt_ann}-per-EMP_TERM"]     = df[amt_ann] - (df[day_bir] - df[day_emp])
df[f"{amt_ann}-per-PUBLISH_TERM"] = df[amt_ann] - (df[day_bir] - df[day_pub])

df[f"{day_bir}-per-EMP_TERM"] = df[day_bir] - (df[day_bir] - df[day_emp])
df[f"{day_bir}-per-PUBLISH_TERM"] = df[day_bir] - (df[day_bir] - df[day_pub])
df[f"{day_bir}-per-REGIST_TERM"] = df[day_bir] - (df[day_bir] - df[day_reg])
df[f"{day_bir}-per-PHONE_TERM"] = df[day_bir] - (df[day_bir] - df[day_pho])


aggs = {}

for col in df.columns:
    if col in org_col_list:
        continue
    aggs[col] = ['sum', 'mean', 'max', 'min', 'std']
    
    
df_agg = df.groupby(key).agg(aggs)
new_col_list = [f"{col}-{method}" for col in aggs.keys() for method in aggs[col]]
df_agg.columns = new_col_list

print(df_agg.shape)
display(df_agg.head())

df_agg = base.join(df_agg)
utils.save_feature(df_feat=df_agg, is_train=2, target=target, ignore_list=[key, target], prefix=prefix)

(356255, 185)


,AMT_CREDIT_per_AMT_ANNUITY-sum,AMT_CREDIT_per_AMT_ANNUITY-mean,AMT_CREDIT_per_AMT_ANNUITY-max,AMT_CREDIT_per_AMT_ANNUITY-min,AMT_CREDIT_per_AMT_ANNUITY-std,AMT_CREDIT_per_AMT_INCOME_TOTAL-sum,AMT_CREDIT_per_AMT_INCOME_TOTAL-mean,AMT_CREDIT_per_AMT_INCOME_TOTAL-max,AMT_CREDIT_per_AMT_INCOME_TOTAL-min,AMT_CREDIT_per_AMT_INCOME_TOTAL-std,AMT_ANNUITY_per_AMT_INCOME_TOTAL-sum,AMT_ANNUITY_per_AMT_INCOME_TOTAL-mean,AMT_ANNUITY_per_AMT_INCOME_TOTAL-max,AMT_ANNUITY_per_AMT_INCOME_TOTAL-min,AMT_ANNUITY_per_AMT_INCOME_TOTAL-std,AMT_CREDIT_per_DAYS_BIRTH-sum,AMT_CREDIT_per_DAYS_BIRTH-mean,AMT_CREDIT_per_DAYS_BIRTH-max,AMT_CREDIT_per_DAYS_BIRTH-min,AMT_CREDIT_per_DAYS_BIRTH-std,AMT_INCOME_TOTAL_per_DAYS_BIRTH-sum,AMT_INCOME_TOTAL_per_DAYS_BIRTH-mean,AMT_INCOME_TOTAL_per_DAYS_BIRTH-max,AMT_INCOME_TOTAL_per_DAYS_BIRTH-min,AMT_INCOME_TOTAL_per_DAYS_BIRTH-std,AMT_ANNUITY_per_DAYS_BIRTH-sum,AMT_ANNUITY_per_DAYS_BIRTH-mean,AMT_ANNUITY_per_DAYS_BIRTH-max,AMT_ANNUITY_per_DAYS_BIRTH-min,AMT_ANNUITY_per_DAYS_BIRTH-std,AMT_CREDIT_per_DAYS_EMPLOYED-sum,AMT_CREDIT_per_DAYS_EMPLOYED-mean,AMT_CREDIT_per_DAYS_EMPLOYED-max,AMT_CREDIT_per_DAYS_EMPLOYED-min,AMT_CREDIT_per_DAYS_EMPLOYED-std,AMT_INCOME_TOTAL_per_DAYS_EMPLOYED-sum,AMT_INCOME_TOTAL_per_DAYS_EMPLOYED-mean,AMT_INCOME_TOTAL_per_DAYS_EMPLOYED-max,AMT_INCOME_TOTAL_per_DAYS_EMPLOYED-min,AMT_INCOME_TOTAL_per_DAYS_EMPLOYED-std,AMT_ANNUITY_per_DAYS_EMPLOYED-sum,AMT_ANNUITY_per_DAYS_EMPLOYED-mean,AMT_ANNUITY_per_DAYS_EMPLOYED-max,AMT_ANNUITY_per_DAYS_EMPLOYED-min,AMT_ANNUITY_per_DAYS_EMPLOYED-std,AMT_CREDIT_per_DAYS_ID_PUBLISH-sum,AMT_CREDIT_per_DAYS_ID_PUBLISH-mean,AMT_CREDIT_per_DAYS_ID_PUBLISH-max,AMT_CREDIT_per_DAYS_ID_PUBLISH-min,AMT_CREDIT_per_DAYS_ID_PUBLISH-std,AMT_INCOME_TOTAL_per_DAYS_ID_PUBLISH-sum,AMT_INCOME_TOTAL_per_DAYS_ID_PUBLISH-mean,AMT_INCOME_TOTAL_per_DAYS_ID_PUBLISH-max,AMT_INCOME_TOTAL_per_DAYS_ID_PUBLISH-min,AMT_INCOME_TOTAL_per_DAYS_ID_PUBLISH-std,AMT_ANNUITY_per_DAYS_ID_PUBLISH-sum,AMT_ANNUITY_per_DAYS_ID_PUBLISH-mean,AMT_ANNUITY_per_DAYS_ID_PUBLISH-max,AMT_ANNUITY_per_DAYS_ID_PUBLISH-min,AMT_ANNUITY_per_DAYS_ID_PUBLISH-std,AMT_CREDIT_per_DAYS_LAST_PHONE_CHANGE-sum,AMT_CREDIT_per_DAYS_LAST_PHONE_CHANGE-mean,AMT_CREDIT_per_DAYS_LAST_PHONE_CHANGE-max,AMT_CREDIT_per_DAYS_LAST_PHONE_CHANGE-min,AMT_CREDIT_per_DAYS_LAST_PHONE_CHANGE-std,AMT_INCOME_TOTAL_per_DAYS_LAST_PHONE_CHANGE-sum,AMT_INCOME_TOTAL_per_DAYS_LAST_PHONE_CHANGE-mean,AMT_INCOME_TOTAL_per_DAYS_LAST_PHONE_CHANGE-max,AMT_INCOME_TOTAL_per_DAYS_LAST_PHONE_CHANGE-min,AMT_INCOME_TOTAL_per_DAYS_LAST_PHONE_CHANGE-std,AMT_ANNUITY_per_DAYS_LAST_PHONE_CHANGE-sum,AMT_ANNUITY_per_DAYS_LAST_PHONE_CHANGE-mean,AMT_ANNUITY_per_DAYS_LAST_PHONE_CHANGE-max,AMT_ANNUITY_per_DAYS_LAST_PHONE_CHANGE-min,AMT_ANNUITY_per_DAYS_LAST_PHONE_CHANGE-std,AMT_CREDIT_per_DAYS_REGISTRATION-sum,AMT_CREDIT_per_DAYS_REGISTRATION-mean,AMT_CREDIT_per_DAYS_REGISTRATION-max,AMT_CREDIT_per_DAYS_REGISTRATION-min,AMT_CREDIT_per_DAYS_REGISTRATION-std,AMT_INCOME_TOTAL_per_DAYS_REGISTRATION-sum,AMT_INCOME_TOTAL_per_DAYS_REGISTRATION-mean,AMT_INCOME_TOTAL_per_DAYS_REGISTRATION-max,AMT_INCOME_TOTAL_per_DAYS_REGISTRATION-min,AMT_INCOME_TOTAL_per_DAYS_REGISTRATION-std,AMT_ANNUITY_per_DAYS_REGISTRATION-sum,AMT_ANNUITY_per_DAYS_REGISTRATION-mean,AMT_ANNUITY_per_DAYS_REGISTRATION-max,AMT_ANNUITY_per_DAYS_REGISTRATION-min,AMT_ANNUITY_per_DAYS_REGISTRATION-std,AMT_ANNUITY_diff_AMT_INCOME_TOTAL-sum,AMT_ANNUITY_diff_AMT_INCOME_TOTAL-mean,AMT_ANNUITY_diff_AMT_INCOME_TOTAL-max,AMT_ANNUITY_diff_AMT_INCOME_TOTAL-min,AMT_ANNUITY_diff_AMT_INCOME_TOTAL-std,DAYS_BIRTH_diff_DAYS_EMPLOYED-sum,DAYS_BIRTH_diff_DAYS_EMPLOYED-mean,DAYS_BIRTH_diff_DAYS_EMPLOYED-max,DAYS_BIRTH_diff_DAYS_EMPLOYED-min,DAYS_BIRTH_diff_DAYS_EMPLOYED-std,DAYS_BIRTH_diff_DAYS_ID_PUBLISH-sum,DAYS_BIRTH_diff_DAYS_ID_PUBLISH-mean,DAYS_BIRTH_diff_DAYS_ID_PUBLISH-max,DAYS_BIRTH_diff_DAYS_ID_PUBLISH-min,DAYS_BIRTH_diff_DAYS_ID_PUBLISH-std,DAYS_BIRTH_diff_DAYS_LAST_PHONE_CHANGE-sum,DAYS_BIRTH_diff_DAYS_LAST_PHONE_CHANGE-mean,D

(307511,) | AMT_CREDIT_per_AMT_ANNUITY-sum
(307511,) | AMT_CREDIT_per_AMT_ANNUITY-mean
(307511,) | AMT_CREDIT_per_AMT_ANNUITY-max
(307511,) | AMT_CREDIT_per_AMT_ANNUITY-min
(307511,) | AMT_CREDIT_per_AMT_ANNUITY-std
(307511,) | AMT_CREDIT_per_AMT_INCOME_TOTAL-sum
(307511,) | AMT_CREDIT_per_AMT_INCOME_TOTAL-mean
(307511,) | AMT_CREDIT_per_AMT_INCOME_TOTAL-max
(307511,) | AMT_CREDIT_per_AMT_INCOME_TOTAL-min
(307511,) | AMT_CREDIT_per_AMT_INCOME_TOTAL-std
(307511,) | AMT_ANNUITY_per_AMT_INCOME_TOTAL-sum
(307511,) | AMT_ANNUITY_per_AMT_INCOME_TOTAL-mean
(307511,) | AMT_ANNUITY_per_AMT_INCOME_TOTAL-max
(307511,) | AMT_ANNUITY_per_AMT_INCOME_TOTAL-min
(307511,) | AMT_ANNUITY_per_AMT_INCOME_TOTAL-std
(307511,) | AMT_CREDIT_per_DAYS_BIRTH-sum
(307511,) | AMT_CREDIT_per_DAYS_BIRTH-mean
(307511,) | AMT_CREDIT_per_DAYS_BIRTH-max
(307511,) | AMT_CREDIT_per_DAYS_BIRTH-min
(307511,) | AMT_CREDIT_per_DAYS_BIRTH-std
(307511,) | AMT_INCOME_TOTAL_per_DAYS_BIRTH-sum
(307511,) | AMT_INCOME_TOTAL_per_DAYS_

In [ ]:
num_list = get_numeric_features(df[org_col_list], ignore_list=ignore_list)
utils.save_feature(df_feat=df[num_list + [key, target]], is_train=2, target=target, ignore_list=[key, target], prefix=prefix)